## Enriching stock market data using Open AI API 

<p align="center">
    <img src="images/nasdaq100.png" width="450">
</p>

The Nasdaq-100 is a stock market index made up of 101 equity securities issued by 100 of the largest non-financial companies listed on the Nasdaq stock exchange. It helps investors compare stock prices with previous prices to determine market performance.

In this project you are provided with two CSV files containing Nasdaq-100 stock information:
- _**nasdaq100_CA.csv**_: contains information about companies in the index such as symbol, name, etc. For this analysis, only companies headquartered in California have been selected.
- _**nasdaq100_price_change.csv**_: contains price changes per stock across periods including (but not limited to) one day, five days, one month, six months, one year, etc.

As an AI developer, you will leverage the OpenAI API to classify companies into sectors and produce a summary of sector and company performance for this year, for the companies in the index that are headquartered in California.

# CSV with Nasdaq-100 stock data

In this project, you have available two CSV files `nasdaq100_CA.csv` and `nasdaq100_price_change.csv`.

## nasdaq100_CA.csv

```py
symbol,name,headQuarter,dateFirstAdded,cik,founded
AAPL,Apple Inc.,"Cupertino, CA",,0000320193,1976-04-01
ABNB,Airbnb,"San Francisco, CA",,0001559720,2008-08-01
ADBE,Adobe Inc.,"San Jose, CA",,0000796343,1982-12-01
...
```

## nasdaq100_price_change.csv

```py
symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
AAPL,-1.7254,-8.30086,-6.20411,3.042,15.64824,42.99992,8.47941,60.96299,245.42031,976.99441,139245.53954
ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.64013,-1.04347,-1.04347,-1.04347,-1.04347
ADBE,0.5409,-1.77817,9.16191,52.0465,38.01522,57.22723,21.96206,17.83037,109.05718,1024.69214,251030.66399
ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.09735,63.42847,92.81874,286.77518,26012.63736
...
```

Link to datacamp: https://www.datacamp.com/datalab/w/78a11016-1f49-4e7a-afe1-23f2ea0b2700/edit

In [1]:
# Start your code here!
import os
import pandas as pd
from openai import OpenAI

# Instantiate an API client
client = OpenAI()

# Continue coding here
# Use as many cells as you like

In [2]:
# Instantiate an API client
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Read in the two datasets
nasdaq100_ca = pd.read_csv("nasdaq100_CA.csv")
price_change = pd.read_csv("nasdaq100_price_change.csv")

In [3]:
# Add symbol into nasdaq100_ca
nasdaq100_ca = nasdaq100_ca.merge(price_change[["symbol", "ytd"]], on="symbol", how="inner")

# Preview the combined dataset
nasdaq100_ca.head()

,symbol,name,headQuarter,dateFirstAdded,cik,founded,ytd
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,42.99992
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,68.66902
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,57.22723
3,ADSK,Autodesk,"San Rafael, CA",NaN,769397,1982-01-30,10.02701
4,AMAT,Applied Materials,"Santa Clara, CA",NaN,6951,1967-11-10,55.46366


In [4]:
# Loop through the NASDAQ companies
for company in nasdaq100_ca["symbol"]:
    # Create a prompt to enrich nasdaq100_ca using OpenAI
    prompt = f'''Classify company {company} into one of the following sectors. Answer only with the sector name: Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, Communication, Energy, Consumer Defensive, Real Estate, Financial.
'''
    # Create a request to the completions endpoint
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )
    # Store the output as a variable called sector
    sector = response.choices[0].message.content
    
    # Add the sector for the corresponding company
    nasdaq100_ca.loc[nasdaq100_ca["symbol"] == company, "Sector"] = sector
    
# Count the number of sectors
nasdaq100_ca["Sector"].value_counts()

Technology            28
Healthcare             5
Real Estate            1
Consumer Defensive     1
Communication          1
Financial              1
Consumer Cyclical      1
Name: Sector, dtype: int64

In [5]:
# Prompt to get stock recommendations
prompt = f'''Provide summary information about Nasdaq-100 stock performance year to date (YTD) of companies headquartered in CA, recommending the two best sectors and two or more companies per sector.
            Company data: {nasdaq100_ca} 
'''

# Get the model response
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )

# Store the output as a variable and print the recommendations
stock_recommendations = response.choices[0].message.content
print(stock_recommendations)

The Nasdaq-100 stock performance year to date (YTD) for companies headquartered in California shows that the Technology sector has been performing well, with companies like Apple Inc. (AAPL), Adobe Inc. (ADBE), Advanced Micro Devices Inc. (AMD), and Nvidia (NVDA) showing significant gains. The Real Estate sector, represented by Airbnb (ABNB), has also performed strongly.

The two best sectors based on YTD performance are Technology and Real Estate. In the Technology sector, Nvidia (NVDA) and Meta Platforms (META) have shown exceptional growth. In the Real Estate sector, Airbnb (ABNB) has also performed well.

Overall, the California-based companies in the Nasdaq-100 index have shown strong performance YTD, with the Technology sector leading the way.
